<a target="_blank" href="https://colab.research.google.com/github/ZHAW-ZAV/TSO-FS25-students/blob/main/01_python_basics/01_03_intro_pandas.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import sys
import os

IN_COLAB = "google.colab" in sys.modules

file_id_co2 = "1XsuqMub8M3SdP50X1MIwAci-lsYS0X9t"
file_id_countries = "17CMqyq43tBbMJ-TId4TH_GF2KskyIUPP"

if IN_COLAB:
    path_to_co2 = "/content/data/co2_emmissions_by_state_2023.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_co2}" -O "{path_to_co2}"

    path_to_countries = "/content/data/icao_country.json"
    os.makedirs(os.path.dirname(path_to_countries), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_countries}" -O "{path_to_countries}"
else:
    import gdown

    url = f"https://drive.google.com/uc?id={file_id_co2}"
    path_to_co2 = "data/co2_emmissions_by_state_2023.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    gdown.download(url, path_to_co2, quiet=False)

    url = f"https://drive.google.com/uc?id={file_id_countries}"
    path_to_countries = "data/icao_country.json"
    os.makedirs(os.path.dirname(path_to_countries), exist_ok=True)
    gdown.download(url, path_to_countries, quiet=False)


The code above loads the data, don't modify.

---------------

***Notebook starts here***

# Introduction to Pandas


## Importing Pandas

In [74]:
import pandas as pd

## Reading Data

In [75]:
# You can read all kinds of files with pandas, not just csv,
# but also excel, json, etc. with pd.read_excel, pd.read_json
co2 = pd.read_csv(path_to_co2)

## Exploring Data

In [ ]:
print(f"The DataFrame has {len(co2)} rows.")
print(f"The columns in the DataFrame are: {co2.columns}")

In [ ]:
# Let's take a look at the first few rows
co2.head()

In [ ]:
# Let's take a look at the last few rows
co2.tail()

In [ ]:
# data types of the columns
co2.info()

In [ ]:
# Some descriptive statistics
co2.describe()

In [ ]:
# Let's see the unique values in the "STATE_NAME" column
co2["STATE_NAME"].unique()

In [ ]:
# How many unique values are there in the "STATE_NAME" column?
len(co2["STATE_NAME"].unique())

In [ ]:
# How many values are there per "STATE_NAME"?
co2["STATE_NAME"].value_counts()

## Filtering and Slicing Data

In [ ]:
# classical approach
co2_ch = co2[co2["STATE_NAME"] == "SWITZERLAND"]
co2_ch

In [ ]:
# using query()
co2_country = co2.query("STATE_NAME == 'SWITZERLAND'")
co2_country

In [ ]:
# with query, you can use variables and complex expressions
country = "ESTONIA"
co2_country = co2.query("STATE_NAME == @country and MONTH <= 6")
co2_country

In [ ]:
# show only the columns we are interested in
#
# note the nested square brackets. The first one is to
# access the columns of the dataframe, e.g. co2["STATE_NAME"].
# The second one creates a list, e.g. ["STATE_NAME", "MONTH", "CO2_QTY_TONNES"].
co2[["STATE_NAME", "MONTH", "CO2_QTY_TONNES"]]

## Aggregating and Sorting Data

In [ ]:
print(f"Total CO2 emissions: {co2['CO2_QTY_TONNES'].sum()}")
print(f"Mean CO2 emissions: {co2['CO2_QTY_TONNES'].mean()}")

# let's display this neatly
print(f"Total CO2 emissions: {co2['CO2_QTY_TONNES'].sum():.0f}")
print(f"Mean CO2 emissions: {co2['CO2_QTY_TONNES'].mean():.1f}")


### Grouping data, single aggregation

In [ ]:
# group by country and sum the CO2 emissions

# this is a bit more advanced, let's break it down
# 1. group by country: co2.groupby("STATE_NAME")
# 2. select the column we are interested in: ["CO2_QTY_TONNES"]
# 3. sum the values: .sum()

co2_country = co2.groupby("STATE_NAME")["CO2_QTY_TONNES"].sum()
co2_country

In [ ]:
# same as above, but sorted
#
# Hint: you can split the code into multiple lines to make it more readable,
#       just wrap the code in parentheses

co2_country = (
    co2.groupby("STATE_NAME")["CO2_QTY_TONNES"].sum().sort_values(ascending=False)
)
co2_country

In [ ]:
# How many values are there per "STATE_NAME"?
# We did this already with this:
# co2["STATE_NAME"].value_counts()
#
# An alternative approach is to use groupby and count the number of rows
# in each group. This is done with the size() function.
print(co2.groupby("STATE_NAME").size())

# or with groupby and count()
print(co2.groupby("STATE_NAME")["STATE_NAME"].count())

### Grouping data, multiple aggregations

In [ ]:
# you can also use the .agg() method to apply multiple functions at once
# for example, we want the sum of the total traffic (TF) and the sum and mean of the CO2 emissions
#
# 1. group by country: co2.groupby("STATE_NAME")
# 2. use the .agg() method to apply multiple functions at once
#    it takes a dictionary as an argument, where the keys are the column names
#    and the values are the functions to apply
# 3. sort the values by the sum of the CO2 emissions
co2_country = (
    co2.groupby("STATE_NAME")
    .agg({"TF": "sum", "CO2_QTY_TONNES": ["sum", "mean"]})
    .sort_values(by=("CO2_QTY_TONNES", "sum"), ascending=False)
)
co2_country

## Modifying and Merging DataFrames

In [ ]:
# let's say we want to add a column with the CO2 emissions per movement
# we can do this by dividing the CO2 emissions by the total traffic
co2["CO2_PER_TF"] = co2["CO2_QTY_TONNES"] / co2["TF"]
co2.sort_values("CO2_PER_TF", ascending=False)

In [ ]:
# let's say we want to add some more information about the countries from another file
# first, load the file. This time it's a json file, so we use pd.read_json
# countries = pd.read_json(path_to_countries)
countries = pd.read_json("data/icao_country.json")
countries.head()

***See notes on join in the presentation***

In [ ]:
# now we want to add ISO two character country code to our dataframe. Merging two datasets is a common
# operation and is often called a "join" or "merge". For this to work, we need a common column in both
# datasets. In our case, we have the ICAO code in both dataframes. It is the column "STATE_CODE" in
# the co2 dataframe and "icao_country" in the countries dataframe.
# We can use the .merge() method to join the two.
co2_with_countries = co2.merge(
    countries, left_on="STATE_CODE", right_on="icao_country", how="inner"
)
print(
    f"The original dataframe has {len(co2)} rows, the new one has {len(co2_with_countries)}."
)
co2_with_countries.head()

In [ ]:
# well, it seems that we have fewer rows in the new dataframe. Let's see which countries are missing.

# get the countries in the co2 dataframe
countries1 = co2["STATE_CODE"].unique()

# get the countries in the merged dataframe
countries2 = co2_with_countries["icao_country"].unique()

# find the countries that are in the co2 dataframe but not in the countries dataframe
# this can easily be done with sets, which are a data type in python
missing_countries = set(countries1) - set(countries2)
missing_countries

# let's see the rows where the country is missing
co2.query("STATE_CODE in @missing_countries")

In [ ]:
# let's try again, this time with a left join
co2_with_countries = co2.merge(
    countries, left_on="STATE_CODE", right_on="icao_country", how="left"
)
print(
    f"The original dataframe has {len(co2)} rows, the new one has {len(co2_with_countries)}."
)
co2_with_countries.head()